<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/rclone_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Install**


In [ ]:
# Cài đặt rClone
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0   8887      0 --:--:-- --:--:-- --:--:--  8898
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Updating index cache for path `/usr/share/man/man1'. Wait...done.
Checking for stray cats under /usr/share/man...
Checking for stray

In [ ]:
# Danh sách remotes đang có
!rclone listremotes

2025/08/22 16:19:27 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults


In [ ]:
# Khôi phục file config
!cp "/content/drive/MyDrive/Colab Notebooks/rclone.conf" /root/.config/rclone/rclone.conf

# ⚙️ **Config remotes**

Modify remotes

In [ ]:
# Tạo mới, chỉnh sửa, xóa hoặc kiểm tra remote.
!rclone config

Backup config

In [ ]:
# Backup config
!cp /root/.config/rclone/rclone.conf /content/rclone.conf
!rclone copy -v "/content/rclone.conf" "/content/drive/MyDrive/Colab Notebooks/backup/rclone-renamed.conf"

# 🎯 **Các lệnh Copy**

In [ ]:
# @title Basic: Copy files between Cloud
# @markdown **Lưu ý**:
# @markdown - nếu thêm `/` ở cuối, chỉ copy nội dung bên trong
# @markdown - không dùng `/` ở cuối thì copy cả folder

source_remote = "onedrive" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive"]
input_folder = "Music/.Lossy/A Sun" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc file, để trống là toàn bộ remote"}
# @markdown ---

destination_remote = "archive" # @param ["archive","onedrive","googledrive2","googledrive1","/content/drive/MyDrive/","/content/"]
destination_folder = "a-sun-320-301022" # @param {"type":"string","placeholder":"dùng `/` để phân cấp folder hoặc tạo thư mục trong archive."}
# @markdown ---

option_flag = "--s3-upload-concurrency 1" # @param {"type":"string","placeholder":"thêm `--flag` nếu cần, dùng space phân cách các flag"}

# Kiểm tra và thực thi lệnh copy
if source_remote and destination_remote:
    # Xử lý đường dẫn
    source_path = f"{source_remote}:{input_folder}" if input_folder else f"{source_remote}:"
    dest_path = f"{destination_remote}:{destination_folder}" if destination_folder else f"{destination_remote}:"

    # Xử lý option_flag
    if option_flag.strip():
        flags = option_flag.strip().split()
        command_flags = "-v " + " ".join(flags)  # Kết hợp -v với các flag người dùng nhập
    else:
        command_flags = "-v"  # Mặc định chỉ dùng -v

    # Chạy lệnh rclone
    !rclone copy {command_flags} "{source_path}" "{dest_path}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote.")

2025/08/22 15:44:31 INFO  : A-Sun - Leave for the moment.mp3: Copied (new)
2025/08/22 15:44:48 INFO  : A-Sun - Safekeeping.mp3: Copied (new)
2025/08/22 15:45:02 INFO  : 
Transferred:   	   65.180 MiB / 97.382 MiB, 67%, 1.265 MiB/s, ETA 25s
Checks:                 0 / 0, -, Listed 8
Transferred:            2 / 8, 25%
Elapsed time:       1m0.0s
Transferring:
 *                             A-Sun - Angel.mp3:100% /10.558Mi, 1.173Mi/s, 0s
 *                           A-Sun - Driving.mp3:  0% /8.878Mi, 0/s, -
 *                      A-Sun - Minor Trauma.mp3:100% /9.374Mi, 39.886Ki/s, 0s
 *                         A-Sun - Stillness.mp3: 71% /9.604Mi, 491.999Ki/s, 5s

2025/08/22 15:45:07 INFO  : A-Sun - Angel.mp3: Copied (new)
2025/08/22 15:45:08 INFO  : A-Sun - Driving.mp3: Copied (new)
2025/08/22 15:45:10 INFO  : A-Sun - Stillness.mp3: Copied (new)
2025/08/22 15:45:15 INFO  : A-Sun - Switch.mp3: Copied (new)
2025/08/22 15:45:17 INFO  : A-Sun - Yours World.mp3: Copied (new)
2025/08/22 15:45:2



---



In [ ]:
!rclone lsf "archive:1okrock-liveshow"

1okrock-liveshow.thumbs/
1okrock-liveshow_archive.torrent
1okrock-liveshow_files.xml
1okrock-liveshow_meta.sqlite
1okrock-liveshow_meta.xml
ONE_OK_ROCK_BD_DISC1.ISO
ONE_OK_ROCK_BD_DISC1.mp4
ONE_OK_ROCK_BD_DISC12.mp4
ONE_OK_ROCK_BD_DISC2.ISO
ONE_OK_ROCK_BD_DISC2.mp4
ONE_OK_ROCK_BD_DISC22.mp4
__ia_thumb.jpg


In [ ]:
!rclone copy -v "https://data.terabox.app/file/355a8f3d0d448004bfd44760355e9e24?bkt=en-07c9b0a504a3706095b5bed33474fa845279f58cd625ea8c91609d778afe1145613e6382e58cd8cd&xcode=df67ed55c6204494b55f5ec8e654e0c13e79c2291fa72c2d874035aa226821b9059abe7c8628492e4f74607fb2ca4215c8167d68cbed08ac&fid=4399953669262-250528-1045094404813338&time=1755879512&sign=FDTAXUGERLQlBHSKfWaon-DCb740ccc5511e5e8fedcff06b081203-GnmoxutjLrn2n9uJXUDTZH2At0s%3D&to=139&size=14419252790&sta_dx=14419252790&sta_cs=6&sta_ft=rar&sta_ct=7&sta_mt=0&fm2=MH%2Ctky%2CAnywhere%2C%2CSGFub2k%3D%2Cany&region=tky&ctime=1721389741&mtime=1755879490&resv0=-1&resv1=0&resv2=&resv3=&resv4=14419252790&vuk=4399953669262&iv=0&htype=&randtype=&newver=1&newfm=1&secfm=1&flow_ver=3&pkey=en-60a4ed0a949c71f4b2eda5effa64b454a071fd3e07b901278f4021729bdc94457bd6b613a0460ebb&sl=68091977&expires=1755908312&rt=pr&r=586159288&vbdid=-&fin=%5BHaDoanTV.Com%5DLISCFSVH.rar&fn=%5BHaDoanTV.Com%5DLISCFSVH.rar&rtype=1&dp-logid=8886183457023662952&dp-callid=0.1&hps=1&tsl=2000&csl=2000&fsl=-1&csign=Hxn20zAbm7kxnk88YOPGGnuyJYc%3D&so=0&ut=6&uter=4&serv=0&uc=3706416523&ti=9c74a385a1d3afc573a68b6d53825de85e1c8a5eaa922718&tuse=&raw_appid=0&ogr=0&rregion=XVVi&adg=&reqlabel=250528_f_01ef7f5e1a9c16c0008388deeb54fe5a_-1_db7dff196c8e03a1b341f2cd746c4229&ccn=VN&by=themis" "/content/torrent_download/"

2025/08/22 16:21:09 CRITICAL: Failed to create file system for "https://data.terabox.app/file/355a8f3d0d448004bfd44760355e9e24?bkt=en-07c9b0a504a3706095b5bed33474fa845279f58cd625ea8c91609d778afe1145613e6382e58cd8cd&xcode=df67ed55c6204494b55f5ec8e654e0c13e79c2291fa72c2d874035aa226821b9059abe7c8628492e4f74607fb2ca4215c8167d68cbed08ac&fid=4399953669262-250528-1045094404813338&time=1755879512&sign=FDTAXUGERLQlBHSKfWaon-DCb740ccc5511e5e8fedcff06b081203-GnmoxutjLrn2n9uJXUDTZH2At0s%3D&to=139&size=14419252790&sta_dx=14419252790&sta_cs=6&sta_ft=rar&sta_ct=7&sta_mt=0&fm2=MH%2Ctky%2CAnywhere%2C%2CSGFub2k%3D%2Cany&region=tky&ctime=1721389741&mtime=1755879490&resv0=-1&resv1=0&resv2=&resv3=&resv4=14419252790&vuk=4399953669262&iv=0&htype=&randtype=&newver=1&newfm=1&secfm=1&flow_ver=3&pkey=en-60a4ed0a949c71f4b2eda5effa64b454a071fd3e07b901278f4021729bdc94457bd6b613a0460ebb&sl=68091977&expires=1755908312&rt=pr&r=586159288&vbdid=-&fin=%5BHaDoanTV.Com%5DLISCFSVH.rar&fn=%5BHaDoanTV.Com%5DLISCFSVH.rar&rtype

### 🍷 **Ghi chú**

**Command**
* `copy` : Copy file/thư mục từ nguồn tới đích, bỏ qua file giống nhau
* `check` : So sánh nội dung giữa nguồn và đích
* `sync` : đồng bộ (dùng --dry-run kiểm tra trước khi chạy thật).
* `ls`: Liệt kê file và kích thước (mặc định).
* `lsf`: Chỉ hiển thị tên file/thư mục.
* `lsd`: Chỉ hiển thị tên thư mục con.
* `size`: Hiển thị tổng kích thước của thư mục.

---
**Flag**

* Ghi đè nếu nguồn mới hơn: `--update`. (không dùng được trên archive)
* Bỏ qua các file đã tồn tại: `--ignore-existing`. (phù hợp với *archive.org* để tránh gây lỗi, còn lại thì dùng mặc định `copy` là đủ).
*
 * Chọn số luồng copy (nếu không dùng thì mặc định là 4)): `--transfers 8`.
 * Dùng cho S3 của Amazon: `--s3-upload-concurrency 1`. (archive.org file lớn ưu tiên dùng ✅ 1 luồng, tối đa dùng được 2 luồng).

* `--dry-run` : Chạy thử, kiểm tra trước.

* `--include "*.mp4"` để chỉ copy file .mp4.
* `--exclude "*.tmp`" để bỏ qua file tạm.
 * Ex lọc files: `!rclone ls "archive:cd-luong-bich-huu" --include "*.flac"`
 * Ex 2: `rclone copy source dest --include "*.jpg" --exclude "*.gif"`

* `--error string` : Báo cáo files lỗi

# 👓 **Xem Nội Dung Remote**

In [ ]:
# @markdown ### Hướng dẫn:
# @markdown - **List Mode**: Chọn cách hiển thị nội dung:
# @markdown   - `ls`: Liệt kê file và kích thước (mặc định).
# @markdown   - `lsf`: Chỉ hiển thị tên file/thư mục.
# @markdown   - `lsd`: Chỉ hiển thị tên thư mục con.
# @markdown   - `size`: Hiển thị tổng kích thước của thư mục.

list_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/","/content/","googledrive1","onedrive","googledrive2","archive"]
input_folder = "tinhhe-lontien-thyetmin/" # @param {"type":"string"}
list_mode = "lsf" # @param ["lsf","lsd","ls","size"]
# @markdown 🔎 Lọc file:
file_type = "mkv" # @param {"type":"string"}
file_type_sum = "Default" # @param ["Default", "Sum"]

# Xử lý file_type_sum thành lệnh phù hợp
sum_command = "" if file_type_sum == "Default" else " | wc -l"


!rclone {list_mode} "{list_remote}:{input_folder}"  | grep ".{file_type}"  {sum_command}


# ======================================
# grep ".mp4": Lọc các dòng có chứa .mp4.
# wc -l: Đếm số dòng (tức là số file .mp4).

# 🗃️ **.Rar .Zip .7zip**

## Rar

In [ ]:
# Install unrar
!apt-get install unrar

In [ ]:
# Giải nén với password (ex: DDDTT)
!unrar x -pdomdom2010 /content/copy_temp/*.zip /content/copy_temp/extracted/

## Zip

In [ ]:
# Install unzip
!apt-get install -y unzip

In [ ]:
# Giải nén file .zip với password (nếu có)
!unzip -P kirakira /content/copy_temp/Ed01.zip -d /content/copy_temp/extracted/

In [ ]:
# Dùng vòng lặp `for` Giải nén nhiều file .zip trong thư mục với password (nếu có)
import os
for zip_file in os.listdir("/content/copy_temp/2/"):
    if zip_file.endswith(".zip"):
        !unzip -P domdom2010 "/content/copy_temp/2/{zip_file}" -d /content/copy_temp/2/s2/
        print(f"Đã giải nén {zip_file}")

---
## uncheck

rar

In [ ]:
# hình như là nén files, hoặc ghép files. cần thử lại.
!cat /content/torrent_download/HI1\ DDDTT.part*.rar > /content/HI1_DDDTT_full.rar

In [ ]:
# Split file nén
!split -b 10G /content/HI1_DDDTT_full.rar /content/HI1_DDDTT_part_
# copy file trùng tên, chhia theo thứ tự.
!rclone copy /content/HI1_DDDTT_part_* archive:tinhhe-lontien-thyetmin/1990/ -v

7zip
* p7zip-full hỗ trợ hầu hết định dạng (.7z, .zip, .rar, .iso, .tar, .gzip, v.v.), nhưng hiệu suất giải nén .rar tốt hơn với unrar.

In [ ]:
!apt-get update
!apt-get install -y p7zip-full

In [ ]:
!7z x /content/file.7z -o/content/extracted/ -pYOUR_PASSWORD

In [ ]:
!7z x /content/file.iso -o/content/extracted/

In [ ]:
import os

# Đường dẫn thư mục chứa file nén
source_folder = "/content/copy_temp/2/"
extract_folder = "/content/copy_temp/2/zip2/"
password = "domdom2010"  # Thay bằng password thực tế

# Tạo thư mục đích nếu chưa có
!mkdir -p {extract_folder}

# Lặp qua các file trong thư mục
for filename in os.listdir(source_folder):
    if filename.endswith(".zip"):
        file_path = os.path.join(source_folder, filename)
        # Sử dụng 7z để giải nén
        !7z x "{file_path}" -o"{extract_folder}" -p{password} -y
        if os.path.exists(os.path.join(extract_folder, filename.replace(".zip", ""))):
            print(f"Đã giải nén {filename} thành công")
        else:
            print(f"Lỗi: Không thể giải nén {filename}. Kiểm tra file hoặc password!")

cai đặt cả 3

In [ ]:
# Cài đặt các công cụ giải nén
!apt-get update
!apt-get install -y unrar unzip p7zip-full

# 📺 **yt-dlp**

## ⚙️ Install yt-dlp & ffmpeg
* ⚠️Chú ý dùng ! trước command
* Nếu lỗi 403 Forbidden, thêm --geo-bypass

In [ ]:
pip install -U yt-dlp

In [ ]:
!sudo apt install ffmpeg

## ⏬ Hướng dẫn


*   `!yt-dlp`: không dùng flag thì tự chọn định dạng tốt nhất
*   `-v`: để xem log
*   Subtitle:
 *   `--write-subs --sub-lang en`: tải 1 hoặc nhiều subtitles `en,vi,kr`. (ex: *en = english*)
 *   `--all-subs`: tải file và tất cả subtitles.
 *   `--list-subs`: danh sách các subtitles.
 *   `--skip-download`: bỏ qua không tải files, nếu chỉ tải subtitle.

*   `-F`: xem tất cả định dạng
*   `-f`: chọn tải định dạng (ex: `-f 251`)

*   `--get-urlurl`: tải .m3u8

*   `-o`: đặt tên file.
*   `-P`: path đường dẫn để lưu file.

---
`--postprocessor-args "-ss 00:00 -to 00:55"`: cắt âm thanh <~ chua test


In [ ]:
!yt-dlp --all-subs "https://wetv.vip/vi/play/87t5rpu3kbh1qm8/" -o "Xin_Hay_Yeu_Em_E7" -P "Dame na Watashi ni Koishite Kudasai 2016"

# 🚄 Torrent


*   ⚠️ Không nên dùng vì vi phạm chính sách Colab



In [ ]:
# @title 📌 Install torrent
#  install
!pip install lbry-libtorrent

In [ ]:
# @title Good
import os, time, libtorrent as lt

SAVE_PATH = "/content/torrent_download"
os.makedirs(SAVE_PATH, exist_ok=True)

def display_progress(h):
    s = h.status()
    state_str = [
        'queued', 'checking', 'downloading metadata', 'downloading',
        'finished', 'seeding', 'allocating', 'checking fastresume'
    ]
    state = state_str[s.state] if 0 <= s.state < len(state_str) else f"state:{s.state}"
    return (f"Tiến độ: {s.progress*100:.2f}% | Down: {s.download_rate/1024:.1f} KB/s | "
            f"Up: {s.upload_rate/1024:.1f} KB/s | Peer: {s.num_peers} | {state}")

def list_files(h):
    ti = h.get_torrent_info()
    fs = ti.files()
    n = fs.num_files()
    return [(i, fs.file_path(i), fs.file_size(i)) for i in range(n)]

def file_progress_bytes(h):
    # v2: trả về list bytes đã tải cho từng file
    try:
        return h.file_progress()
    except TypeError:
        return h.file_progress(0)

# --- session ---
ses = lt.session()
try:
    ses.listen_on(6881, 6891)
except Exception:
    pass
ses.start_dht()

# --- add magnet ---
magnet = input("Nhập magnet (q để thoát): ").strip()
if magnet.lower() == 'q':
    raise SystemExit

# thêm flag sequential cho ổn định streaming (tùy chọn)
atp = lt.parse_magnet_uri(magnet)
atp.save_path = SAVE_PATH
if hasattr(lt.torrent_flags, "sequential_download"):
    atp.flags |= lt.torrent_flags.sequential_download

# (tùy chọn) thêm tracker để bắt metadata nhanh
atp.trackers = [
    "udp://tracker.opentrackr.org:1337/announce",
    "udp://open.stealth.si:80/announce",
    "udp://tracker.openbittorrent.com:6969/announce",
]

h = ses.add_torrent(atp)

# --- chờ metadata ---
t0 = time.time()
while not h.has_metadata():
    if time.time() - t0 > 90:
        print("Timeout lấy metadata (ít peer/seeder).")
        raise SystemExit
    time.sleep(1)
print("Đã lấy metadata.")

# --- liệt kê file ---
files = list_files(h)
for i, p, sz in files:
    print(f"[{i}] {p} ({sz/1024/1024:.2f} MB)")

# --- chọn tải ---
choice = input("Tải toàn bộ (y) hay chọn file (n)? ").strip().lower()
selected = None
if choice == 'n':
    raw = input("Nhập index, ví dụ 0,2,4: ")
    idxs = []
    for x in raw.split(','):
        x = x.strip()
        if x.isdigit() and 0 <= int(x) < len(files):
            idxs.append(int(x))
    selected = sorted(set(idxs))

# --- đặt ưu tiên file ---
num_files = len(files)
if selected:
    prio = [0]*num_files
    for i in selected: prio[i] = 1
else:
    prio = [1]*num_files

try:
    h.prioritize_files(prio)
except AttributeError:
    for i, p in enumerate(prio):
        h.file_priority(i, p)

print("Bắt đầu tải...")
last = 0
while True:
    s = h.status()
    now = time.time()
    if now - last >= 5:
        print(display_progress(h))
        if selected:
            prog = file_progress_bytes(h)
            for i in selected:
                done = prog[i] if i < len(prog) else 0
                size = files[i][2]
                pct = (done/size*100) if size else 0
                print(f"  - [{i}] {files[i][1]}: {pct:.2f}% ({done/1024/1024:.2f}/{size/1024/1024:.2f} MB)")
        last = now
    if s.state in (lt.torrent_status.seeding, lt.torrent_status.finished):
        print("Hoàn tất.")
        break
    time.sleep(1)

# liệt kê file kết quả
for root, _, fns in os.walk(SAVE_PATH):
    for f in fns:
        p = os.path.join(root, f)
        print("- ", p)


/tmp/ipython-input-1886093257.py:32: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)
/tmp/ipython-input-1886093257.py:35: DeprecationWarning: start_dht() is deprecated
  ses.start_dht()


Nhập magnet (q để thoát): magnet:?xt=urn:btih:39bd1496f0e677b26a231073f5ffb7259e256bff&dn=Photoshop.CC.2018.x64&xl=1726530603&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.publicbt.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.ccc.de%3A80%2Fannounce&tr=http%3A%2F%2Frutracker.ru%2Fbt%2Fannounce.php%3Fuk%3DzK10kZwB06&tr=http%3A%2F%2Fretracker.local%2Fannounce


/tmp/ipython-input-1886093257.py:59: DeprecationWarning: has_metadata() is deprecated
  while not h.has_metadata():
/tmp/ipython-input-1886093257.py:17: DeprecationWarning: get_torrent_info() is deprecated
  ti = h.get_torrent_info()


Đã lấy metadata.
[0] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.iso (1646.54 MB)
[1] Photoshop.CC.2018.x64/m0nkrus.nfo (0.00 MB)
[2] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.sfv (0.00 MB)
[3] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.sha (0.00 MB)
[4] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.md5 (0.00 MB)
Tải toàn bộ (y) hay chọn file (n)? y
Bắt đầu tải...
Tiến độ: 1.60% | Down: 2267.3 KB/s | Up: 70.3 KB/s | Peer: 1 | downloading
Tiến độ: 1.78% | Down: 1226.3 KB/s | Up: 37.4 KB/s | Peer: 1 | downloading
Tiến độ: 2.49% | Down: 2087.0 KB/s | Up: 64.5 KB/s | Peer: 1 | downloading
Tiến độ: 2.69% | Down: 1256.6 KB/s | Up: 38.4 KB/s | Peer: 1 | downloading
Tiến độ: 3.36% | Down: 2266.2 KB/s | Up: 70.1 KB/s | Peer: 1 | downloading
Tiến độ: 3.68% | Down: 1448.3 KB/s | Up: 44.4 KB/s | Peer: 1 | downloading
Tiến độ: 4.34% | Down: 1886.6 KB/s | Up: 58.3 KB/s | Peer: 1 | downloading
Tiến độ: 5.04% | Do

In [ ]:
# @title da tai duoc
import os, time, libtorrent as lt

SAVE_PATH = "/content/torrent_download"
os.makedirs(SAVE_PATH, exist_ok=True)

def display_progress(h):
    s = h.status()
    state_str = [
        'queued', 'checking', 'downloading metadata', 'downloading',
        'finished', 'seeding', 'allocating', 'checking fastresume'
    ]
    state = state_str[s.state] if 0 <= s.state < len(state_str) else f"state:{s.state}"
    return (f"Tiến độ: {s.progress*100:.2f}% | Down: {s.download_rate/1024:.1f} KB/s | "
            f"Up: {s.upload_rate/1024:.1f} KB/s | Peer: {s.num_peers} | {state}")

def list_files(h):
    ti = h.get_torrent_info()
    fs = ti.files()
    n = fs.num_files()
    return [(i, fs.file_path(i), fs.file_size(i)) for i in range(n)]

def file_progress_bytes(h):
    # v2: trả về list bytes đã tải cho từng file
    try:
        return h.file_progress()
    except TypeError:
        return h.file_progress(0)

# --- session ---
ses = lt.session()
try:
    ses.listen_on(6881, 6891)
except Exception:
    pass
ses.start_dht()

# --- add magnet ---
magnet = input("Nhập magnet (q để thoát): ").strip()
if magnet.lower() == 'q':
    raise SystemExit

# thêm flag sequential cho ổn định streaming (tùy chọn)
atp = lt.parse_magnet_uri(magnet)
atp.save_path = SAVE_PATH
if hasattr(lt.torrent_flags, "sequential_download"):
    atp.flags |= lt.torrent_flags.sequential_download

# (tùy chọn) thêm tracker để bắt metadata nhanh
atp.trackers = [
    "udp://tracker.opentrackr.org:1337/announce",
    "udp://open.stealth.si:80/announce",
    "udp://tracker.openbittorrent.com:6969/announce",
]

h = ses.add_torrent(atp)

# --- chờ metadata ---
t0 = time.time()
while not h.has_metadata():
    if time.time() - t0 > 90:
        print("Timeout lấy metadata (ít peer/seeder).")
        raise SystemExit
    time.sleep(1)
print("Đã lấy metadata.")

# --- liệt kê file ---
files = list_files(h)
for i, p, sz in files:
    print(f"[{i}] {p} ({sz/1024/1024:.2f} MB)")

# --- chọn tải ---
choice = input("Tải toàn bộ (y) hay chọn file (n)? ").strip().lower()
selected = None
if choice == 'n':
    raw = input("Nhập index, ví dụ 0,2,4: ")
    idxs = []
    for x in raw.split(','):
        x = x.strip()
        if x.isdigit() and 0 <= int(x) < len(files):
            idxs.append(int(x))
    selected = sorted(set(idxs))

# --- đặt ưu tiên file ---
num_files = len(files)
if selected:
    prio = [0]*num_files
    for i in selected: prio[i] = 1
else:
    prio = [1]*num_files

try:
    h.prioritize_files(prio)
except AttributeError:
    for i, p in enumerate(prio):
        h.file_priority(i, p)

print("Bắt đầu tải...")
last = 0
while True:
    s = h.status()
    now = time.time()
    if now - last >= 5:
        print(display_progress(h))
        if selected:
            prog = file_progress_bytes(h)
            for i in selected:
                done = prog[i] if i < len(prog) else 0
                size = files[i][2]
                pct = (done/size*100) if size else 0
                print(f"  - [{i}] {files[i][1]}: {pct:.2f}% ({done/1024/1024:.2f}/{size/1024/1024:.2f} MB)")
        last = now
    if s.state in (lt.torrent_status.seeding, lt.torrent_status.finished):
        print("Hoàn tất.")
        break
    time.sleep(1)

# liệt kê file kết quả
for root, _, fns in os.walk(SAVE_PATH):
    for f in fns:
        p = os.path.join(root, f)
        print("- ", p)


/tmp/ipython-input-1886093257.py:32: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)
/tmp/ipython-input-1886093257.py:35: DeprecationWarning: start_dht() is deprecated
  ses.start_dht()


Nhập magnet (q để thoát): magnet:?xt=urn:btih:39bd1496f0e677b26a231073f5ffb7259e256bff&dn=Photoshop.CC.2018.x64&xl=1726530603&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.publicbt.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.ccc.de%3A80%2Fannounce&tr=http%3A%2F%2Frutracker.ru%2Fbt%2Fannounce.php%3Fuk%3DzK10kZwB06&tr=http%3A%2F%2Fretracker.local%2Fannounce


/tmp/ipython-input-1886093257.py:59: DeprecationWarning: has_metadata() is deprecated
  while not h.has_metadata():
/tmp/ipython-input-1886093257.py:17: DeprecationWarning: get_torrent_info() is deprecated
  ti = h.get_torrent_info()


Đã lấy metadata.
[0] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.iso (1646.54 MB)
[1] Photoshop.CC.2018.x64/m0nkrus.nfo (0.00 MB)
[2] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.sfv (0.00 MB)
[3] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.sha (0.00 MB)
[4] Photoshop.CC.2018.x64/Adobe.Photoshop.CC.2018.u2.x64.Multilingual.md5 (0.00 MB)
Tải toàn bộ (y) hay chọn file (n)? y
Bắt đầu tải...
Tiến độ: 1.60% | Down: 2267.3 KB/s | Up: 70.3 KB/s | Peer: 1 | downloading
Tiến độ: 1.78% | Down: 1226.3 KB/s | Up: 37.4 KB/s | Peer: 1 | downloading
Tiến độ: 2.49% | Down: 2087.0 KB/s | Up: 64.5 KB/s | Peer: 1 | downloading
Tiến độ: 2.69% | Down: 1256.6 KB/s | Up: 38.4 KB/s | Peer: 1 | downloading
Tiến độ: 3.36% | Down: 2266.2 KB/s | Up: 70.1 KB/s | Peer: 1 | downloading
Tiến độ: 3.68% | Down: 1448.3 KB/s | Up: 44.4 KB/s | Peer: 1 | downloading
Tiến độ: 4.34% | Down: 1886.6 KB/s | Up: 58.3 KB/s | Peer: 1 | downloading
Tiến độ: 5.04% | Do

In [ ]:
# torrent download full

import libtorrent as lt
import time

ses = lt.session()
ses.listen_on(6881, 6891)
params = {
    'save_path': '/content/torrent_download',
    'storage_mode': lt.storage_mode_t(2)
}
# Thay link magnet ở trong "" dòng dưới 👇
link = "magnet:?xt=urn:btih:3f45046a38f729a4443441e74b23f8cc8f8432e2&dn=Little.Witch.in.the.Woods.v3.0.6.0b.rar&xl=252463291&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.internetwarriors.net%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fcoppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce"
handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print("Đang tải torrent...")

while not handle.has_metadata():
    time.sleep(1)
print("Đã lấy metadata, bắt đầu tải...")

while handle.status().state != lt.torrent_status.seeding:
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating']
    print(f"Tiến độ: {s.progress * 100:.2f}% - Tốc độ: {s.download_rate / 1000:.2f} kB/s - Trạng thái: {state_str[s.state]}")
    time.sleep(5)
else:
    print("Hoàn tất tải torrent!")

# Tải audio (mp3) cho 5 link cùng lúc

*   Chỉ cần thay LINK_1 … LINK_5 bằng 5 link của bạn (có thể dán xuống dòng cũng được).

```
!yt-dlp -x --audio-format m4a --audio-quality 0 \
  --embed-thumbnail --add-metadata \
  -o "%(title)s.%(ext)s" \
  "LINK_1" "LINK_2" "LINK_3" "LINK_4" "LINK_5"
  -P "/content/drive/MyDrive/YoutubeAudio"
```




JavaScript giữ kết nối colab


```
# F12 > Console
function ClickConnect(){
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 300000); // 5 phút
```

